In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Oct_weekend

No dicts found; please check load_dicts...


In [2]:
xi_list = GLS_Oct_weekend.xi_list
P = GLS_Oct_weekend.P
L = GLS_Oct_weekend.L  # dimension of xi

314

In [3]:
size(P, 1), size(P, 2)

(56,314)

In [4]:
using JuMP

mGLSOctia = Model()

@defVar(mGLSOctia, lam[1:size(P,1)] >= 0)

@defVar(mGLSOctia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @addConstraint(mGLSOctia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @addNLConstraint(mGLSOctia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @addNLConstraint(mGLSOctia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@setNLObjective(mGLSOctia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSOctia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    70022
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    52752

Total number of variables............................:    17640
                     variables with only lower bounds:    17640
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [5]:
getValue(lam)

56-element Array{Float64,1}:
 391.878
 595.514
 259.707
 352.528
 354.2  
 338.753
 225.662
 261.911
 304.505
 228.327
 397.29 
 438.491
 464.069
   ⋮    
 309.115
 267.243
 379.428
 244.579
 211.126
 251.786
 213.813
 321.659
 399.71 
 236.459
 268.11 
 335.187

In [6]:
getObjectiveValue(mGLSOctia)

1.0000000000006115

In [7]:
GLS_Oct_weekend.saveDemandVec(getValue(lam))